## Oracle Database 23c AI Vector Search

In [ ]:
from os import getenv
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.oci_generative_ai import OCIGenAI
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser

import oracledb

load_dotenv()

COMPARTMENT_ID = getenv("COMPARTMENT_ID")
SERVICE_ENDPOINT = getenv("GEN_AI_INFERENCE_ENDPOINT") # https://inference.generativeai.us-chicago-1.oci.oraclecloud.com
UN = getenv('DB_USERNAME')
PW = getenv('DB_PASSWORD')
DSN = getenv('DB_DSN')
TNS_ADMIN = getenv('TNS_ADMIN')

### Using Oracle Generative AI Service

In [ ]:
# Create demo table
with oracledb.connect(user=UN, password=PW, dsn=DSN, config_dir=TNS_ADMIN) as connection:
    with connection.cursor() as cursor:
        sql = """
            CREATE TABLE IF NOT EXISTS OCI_SERVICES(
                id number primary key,
                title varchar2(256),
                category varchar2(256),
                url varchar2(256),
                text varchar2(1024),
                text_v vector(1024, float32)
            )
        """
        cursor.execute(sql)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json(path_or_buf="../data/oci-services.jsonl", lines=True)
df.head()

In [ ]:
embeddings = OCIGenAIEmbeddings(
    model_id="cohere.embed-english-v3.0",
    service_endpoint=SERVICE_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
    auth_type="INSTANCE_PRINCIPAL"
)

response = embeddings.embed_query(text="This is a query in English.")
print(response)

In [ ]:
sql_template = """
    INSERT INTO OCI_SERVICES(
        id,
        title,
        category,
        url,
        text,
        text_v
    ) VALUES (
        :1,
        :2,
        :3,
        :4,
        :5,
        :6
    )
"""
with oracledb.connect(user=UN, password=PW, dsn=DSN, config_dir=TNS_ADMIN) as connection:
    with connection.cursor() as cursor:
        cursor.setinputsizes(None, oracledb.DB_TYPE_VECTOR)
        for idx, row in df.iterrows():
            text_v = embeddings.embed_query(row['text'])
            cursor.execute(
                sql_template,
                [
                    row['id'],
                    row['title'],
                    row['category'],
                    row['url'],
                    row['text'],
                    f"{text_v}"
                ]
            )
    connection.commit()

In [ ]:
with oracledb.connect(user=UN, password=PW, dsn=DSN, config_dir=TNS_ADMIN) as connection:
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM OCI_SERVICES")
        for row in cursor:
            print(row)

In [ ]:
query = "what is region?"
query_v = embeddings.embed_query(text=query)

context = list()

with oracledb.connect(user=UN, password=PW, dsn=DSN, config_dir=TNS_ADMIN) as connection:
    with connection.cursor() as cursor:
        cursor.execute("""
            SELECT
                ID,
                TITLE,
                CATEGORY,
                URL,
                TEXT
            FROM
                OCI_SERVICES
            ORDER BY VECTOR_DISTANCE(TEXT_V, :query)
            FETCH FIRST 1 ROWS ONLY
        """, query=f"{query_v}")
        columns = [col[0] for col in cursor.description]
        cursor.rowfactory = lambda *args: dict(zip(columns, args))
        for row in cursor:
            print(row)
            context.append(row['TEXT'])
print(context)

In [ ]:
query = "what is region?"
query_v = embeddings.embed_query(text=query)

context = list()

with oracledb.connect(user=UN, password=PW, dsn=DSN, config_dir=TNS_ADMIN) as connection:
    with connection.cursor() as cursor:
        cursor.execute("""
            SELECT
                ID,
                TITLE,
                CATEGORY,
                URL,
                TEXT
            FROM
                OCI_SERVICES
            ORDER BY VECTOR_DISTANCE(TEXT_V, :query)
            FETCH FIRST 1 ROWS ONLY
        """, query=f"{query_v}")
        columns = [col[0] for col in cursor.description]
        cursor.rowfactory = lambda *args: dict(zip(columns, args))
        for row in cursor:
            context.append(row['TEXT'])
        template = """Answer the question based only on the following context:
        {context}

        Question: {query}
        """
        prompt = PromptTemplate(
            template=template,
            input_variables=["context", "query"]
        )
        llm = OCIGenAI(
            auth_type="INSTANCE_PRINCIPAL",
            service_endpoint=SERVICE_ENDPOINT,
            model_id="cohere.command",
            compartment_id=COMPARTMENT_ID,
            model_kwargs={
                'max_tokens': 500
            }
        )
        chain = prompt | llm | StrOutputParser()
        output = chain.invoke({"context": context[0], "query": query})
        print(output)